In [15]:
from copulas.univariate import Univariate
import pandas as pd
import json

In [16]:
dtype = {
    "ED_2Clases": 'category',
    #"clin-reservaCognitiva_escolaridad": 'category',
    #"demo-genero": 'category'
    }

In [17]:
df = pd.read_csv(f'../data/original/Identia_UNED_TLP_preprocessed.csv', dtype=dtype)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   ED_2Clases                          314 non-null    category
 1   clin-reservaCognitiva_escolaridad   314 non-null    int64   
 2   demo-edad                           314 non-null    int64   
 3   demo-genero                         314 non-null    int64   
 4   eval-TLP-CubCorsi-totalDirectos-PD  314 non-null    float64 
 5   eval-TLP-CubCorsi-totalinversos-PD  314 non-null    float64 
 6   eval-TLP-CubCorsi-total-PD          314 non-null    float64 
 7   eval-TLP-FigRey-Duracion-PD         314 non-null    float64 
 8   eval-TLP-FigRey-totalCopia-PD       314 non-null    float64 
 9   eval-TLP-FigRey-totalMemoria-PD     314 non-null    float64 
 10  eval-TLP-Stroop-palabras-PD         314 non-null    float64 
 11  eval-TLP-Stroop-color-PD        

In [19]:
map_distribution = {
    "copulas.univariate.gaussian_kde.GaussianKDE": "gaussian_kde",
    "copulas.univariate.gamma.GammaUnivariate": "gamma",
    "copulas.univariate.beta.BetaUnivariate": "beta",
    "copulas.univariate.gaussian.GaussianUnivariate": "gaussian_kde"
}

In [20]:
# Supongamos que ya tienes tu DataFrame llamado `df`
synth_data = pd.DataFrame()
distributions = {}

for col in df.select_dtypes(include=['float64', 'int64']).columns:
    real_data = df[col]
    univariate = Univariate()
    univariate.fit(real_data)
    synth_data[col] = univariate.sample(len(real_data))
    distributions[col] = map_distribution[univariate.to_dict()['type']]

c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\scipy\stats\_continuous_distns.py:716: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\scipy\stats\_continuous_distns.py:721: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\scipy\stats\_continuous_distns.py:721: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\scipy\stats\_continuous_distns.py:716: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\jgala\uned\tfm\synthe

In [21]:
distributions

{'clin-reservaCognitiva_escolaridad': 'gaussian_kde',
 'demo-edad': 'gaussian_kde',
 'demo-genero': 'gaussian_kde',
 'eval-TLP-CubCorsi-totalDirectos-PD': 'gaussian_kde',
 'eval-TLP-CubCorsi-totalinversos-PD': 'gamma',
 'eval-TLP-CubCorsi-total-PD': 'gaussian_kde',
 'eval-TLP-FigRey-Duracion-PD': 'beta',
 'eval-TLP-FigRey-totalCopia-PD': 'gaussian_kde',
 'eval-TLP-FigRey-totalMemoria-PD': 'gaussian_kde',
 'eval-TLP-Stroop-palabras-PD': 'gaussian_kde',
 'eval-TLP-Stroop-color-PD': 'gaussian_kde',
 'eval-TLP-Stroop-palabrasColor-PD': 'gaussian_kde',
 'eval-TLP-Tavec-1_RI_A1-PD': 'gaussian_kde',
 'eval-TLP-Tavec-10_RL_LP-PD': 'gaussian_kde',
 'eval-TLP-Tavec-11_RCl_LP-PD': 'gaussian_kde',
 'eval-TLP-Tavec-2_RI_A5-PD': 'gaussian_kde',
 'eval-TLP-Tavec-3_RI_AT-PD': 'gaussian_kde',
 'eval-TLP-Tavec-8_RL_CP-PD': 'gaussian_kde',
 'eval-TLP-Tavec-9_RCl_CP-PD': 'gaussian_kde'}

In [22]:
with open('distributions.json', 'w') as f:
   json.dump(distributions, f, indent=4)

In [23]:
constraints = []

for col in df.columns:
    if col == 'ED_2Clases':
        continue
    if col == 'demo-edad':
        # Para la edad, se establece un rango de 18 a 100 años
        my_constraint = {
            'constraint_class': 'ScalarRange',
            'table_name': 'TLP',
            'constraint_parameters': {
                'column_name': col,
                'low_value': 18,
                'high_value': 100,
                'strict_boundaries': False
            }
        }
        constraints.append(my_constraint)
    elif col == 'clin-reservaCognitiva_escolaridad':
        # Para la escolaridad, se establece un rango de 0 a 5
        my_constraint = {
            'constraint_class': 'ScalarRange',
            'table_name': 'TLP',
            'constraint_parameters': {
                'column_name': col,
                'low_value': 0,
                'high_value': 5,
                'strict_boundaries': False
            }
        }
        constraints.append(my_constraint)
    elif col == 'demo-genero':
        # Para el género, se establece un rango de 1 a 2
        my_constraint = {
            'constraint_class': 'ScalarRange',
            'table_name': 'TLP',
            'constraint_parameters': {
                'column_name': col,
                'low_value': 1,
                'high_value': 2,
                'strict_boundaries': False
            }
        }
        constraints.append(my_constraint)
    else:
        min_val = df[col].min()
        max_val = df[col].max()
        my_constraint = {
        'constraint_class': 'ScalarRange',
        'table_name': 'TLP',
        'constraint_parameters': {
            'column_name': col,
            'low_value': min_val,
            'high_value': max_val,
            'strict_boundaries': False
            }
        }
        constraints.append(my_constraint)

# Ver ejemplo de las constraints generadas
for c in constraints:
    print(c)


{'constraint_class': 'ScalarRange', 'table_name': 'TLP', 'constraint_parameters': {'column_name': 'clin-reservaCognitiva_escolaridad', 'low_value': 0, 'high_value': 5, 'strict_boundaries': False}}
{'constraint_class': 'ScalarRange', 'table_name': 'TLP', 'constraint_parameters': {'column_name': 'demo-edad', 'low_value': 18, 'high_value': 100, 'strict_boundaries': False}}
{'constraint_class': 'ScalarRange', 'table_name': 'TLP', 'constraint_parameters': {'column_name': 'demo-genero', 'low_value': 1, 'high_value': 2, 'strict_boundaries': False}}
{'constraint_class': 'ScalarRange', 'table_name': 'TLP', 'constraint_parameters': {'column_name': 'eval-TLP-CubCorsi-totalDirectos-PD', 'low_value': 0.0, 'high_value': 1.0, 'strict_boundaries': False}}
{'constraint_class': 'ScalarRange', 'table_name': 'TLP', 'constraint_parameters': {'column_name': 'eval-TLP-CubCorsi-totalinversos-PD', 'low_value': 0.0, 'high_value': 1.0, 'strict_boundaries': False}}
{'constraint_class': 'ScalarRange', 'table_name'

In [24]:
with open('constraints.json', 'w') as f:
   json.dump(constraints, f, indent=4)